In [ ]:
import torch
import numpy as np
import sympy as sym
from sympy.solvers import solve
from matplotlib import pyplot as plt

In [ ]:
c = sym.Symbol('c', positive=True)
mu = sym.Symbol('mu', positive=True)
theta = sym.Symbol('sigma',positive=True)
theta_tst = sym.Symbol('theta_tst',positive=True)
M = sym.Symbol('d',positive=True)
N = sym.Symbol('N',positive=True)
Ntst = sym.Symbol('Ntst',positive=True)

In [ ]:
# First do c < 1, here mu = mu**2

def sym_lambda_inverse_N(c,mu):
  b = -(1-c+mu)
  a = -c*mu
  cq = 1
  numerator1 = -1*b - sym.sqrt(b ** 2 - 4*a*cq)
  denominator = 2*a
  return numerator1/denominator

def sym_lambda_inverse_M(c,mu):
  return c*sym_lambda_inverse_N(c, c*mu)

def sym_lambda_inverse_square_N(c,mu):
  m = sym_lambda_inverse_N(c,mu)
  num = c*m*m + m
  den = -2*c*mu*m - 1 + c - mu
  return -1*num/den

def sym_lambda_inverse_square_M(c,mu):
  return c*c*sym_lambda_inverse_square_N(c, c*mu) 

def sym_scale(c,mu):
  return 1 - (mu)*sym_lambda_inverse_M(c,mu)

def sym_scale_squared(c,mu):
  return sym_lambda_inverse_M(c, mu) - (mu)*sym_lambda_inverse_square_M(c, mu)

def sym_tnorm(c,mu):
  return 1 - sym_scale(c,mu)*c

def sym_tnorm_data(c,mu,M):
  theta = sym.sqrt(M/c)
  return (theta**2)*(1 - sym_scale(c,mu)*c)

def sym_hnorm(c,mu):
  return c*sym_scale_squared(c,mu)

def sym_knorm(c,mu):
  return sym_lambda_inverse_M(c,mu)

def sym_trace_term(c,mu):
  return sym_lambda_inverse_square_M(c,mu)

def sym_tau(c,mu,theta):
  return 1+(theta**2)*sym_tnorm(c,mu)*sym_knorm(c,mu)

def sym_tau_data(c,mu,M):
  theta = sym.sqrt(M/c)
  return 1+(theta**2)*sym_tnorm(c,mu)*sym_knorm(c,mu)
  
def sym_term1(c,mu,theta):
  return (theta**2)*sym_hnorm(c,mu)/(sym_tau(c,mu,theta)**2)

def sym_term2(c,mu,theta):
  return (theta**4)*(sym_tnorm(c,mu)**2)*(sym_trace_term(c,mu))/(sym_tau(c,mu,theta)**2)

def sym_term1_data(c,mu,M):
  theta = sym.sqrt(M/c)
  return (theta**2)*sym_hnorm(c,mu)/(sym_tau(c,mu,theta)**2)

def sym_term2_data(c,mu,M):
  theta = sym.sqrt(M/c)
  return (theta**4)*(sym_tnorm(c,mu)**2)*(sym_trace_term(c,mu))/(sym_tau(c,mu,theta)**2)

def sym_wnorm(c,mu,theta):
  return sym_term1(c,mu,theta)+sym_term2(c,mu,theta)

def sym_wnorm_data(c,mu,M):
  theta = sym.sqrt(M/c)
  return sym_wnorm(c,mu,theta)/M

def sym_bias(c,mu,theta,thetatst,Ntst):
  return (thetatst**2)/(Ntst*sym_tau(c,mu,theta)**2)

def sym_bias_data(c,mu,M,Ntst):
  thetatst = sym.sqrt(Ntst)
  theta = sym.sqrt(M/c)
  return (thetatst**2)/(Ntst*sym_tau(c,mu,theta)**2)

def sym_gen_error_opt(c,mu,Ntst,M):
  thetatst = sym.sqrt(Ntst)
  theta = optimal_theta_over(c,mu,thetatst,Ntst,M)
  return sym_bias(c,mu,theta,thetatst,Ntst) + sym_wnorm(c,mu,theta)/M

def sym_gen_error(c,mu,Ntst,M):
  thetatst = sym.sqrt(Ntst)
  theta = sym.sqrt(M/c)
  return sym_bias(c,mu,theta,thetatst,Ntst) + sym_wnorm(c,mu,theta)/M

def sym_scale_both_squared(c,mu):
  return sym_scale(c,mu) - (mu)*sym_scale_squared(c,mu) 

def sym_term1_training_error(c,mu,theta):
  return c * sym_scale_both_squared(c,mu)*(theta**2)/(sym_tau(c,mu,theta)**2)

def sym_term2_training_error(c,mu,theta):
  return (sym_tnorm(c,mu)**2) * sym_scale_squared(c,mu)*(theta**4)/(sym_tau(c,mu,theta)**2)
  
def sym_wnorm_train(c,mu,theta):
  return sym_term1_training_error(c,mu,theta) + sym_term2_training_error(c,mu,theta)

def sym_training_trace_term(c,mu,theta):
  return ((theta**2) * c * sym_scale(c,mu) / sym_tau(c,mu,theta)) - sym_trace2(c,mu,theta)

def sym_const_term(c,mu,theta):
  return theta ** 2 / (sym_tau(c,mu,theta)**2)

def sym_trace2(c,mu,theta):
  return c * (theta**4) * sym_knorm(c,mu) * sym_tnorm(c,mu) * sym_scale(c,mu) / (sym_tau(c,mu,theta)**2)

def sym_training_error(c,mu,N,M):
  theta = sym.sqrt(M/c)
  return (sym_const_term(c,mu,theta) + sym_wnorm_train(c,mu,theta) - 2*sym_training_trace_term(c,mu,theta))/N

def optimal_theta(c,mu,thetatst,Ntst,M):
  num = 2 * (thetatst**2 / Ntst) * sym_tnorm(c,mu) * sym_knorm(c,mu) * M - sym_hnorm(c,mu)
  denom = 2 * sym_tnorm(c,mu) ** 2 * sym_trace_term(c,mu) - sym_hnorm(c,mu) * sym_tnorm(c,mu) * sym_knorm(c,mu)
  return num / denom

In [ ]:
# c > 1

def sym_lambda_inverse_M_over(c,mu):
  return sym_lambda_inverse_N(1/c, mu)

def sym_lambda_inverse_square_M_over(c,mu):
  return sym_lambda_inverse_square_N(1/c, mu) 

def sym_scale_over(c,mu):
  return 1 - (mu)*sym_lambda_inverse_N(1/c,mu)

def sym_scale_squared_over(c,mu):
  return sym_lambda_inverse_N(1/c, mu) - (mu)*sym_lambda_inverse_square_N(1/c, mu)

def sym_tnorm_over(c,mu):
  return 1 - sym_scale_over(c,mu)

def sym_hnorm_over(c,mu):
  return sym_scale_squared_over(c,mu)

def sym_knorm_over(c,mu):
  return sym_lambda_inverse_N(1/c,mu)/c + (1-1/c)/(mu)

def sym_trace_term_over(c,mu):
  return sym_lambda_inverse_square_N(1/c,mu)/c + (1-1/c)/(mu**2)

def sym_tau_over(c,mu,theta):
  return 1+(theta**2)*sym_tnorm_over(c,mu)*sym_knorm_over(c,mu)
  
def sym_term1_over(c,mu,theta):
  return (theta**2)*sym_hnorm_over(c,mu)/(sym_tau_over(c,mu,theta)**2)

def sym_term2_over(c,mu,theta):
  return (theta**4)*(sym_tnorm_over(c,mu)**2)*(sym_trace_term_over(c,mu))/(sym_tau_over(c,mu,theta)**2)

def sym_wnorm_over(c,mu,theta):
  return sym_term1_over(c,mu,theta)+sym_term2_over(c,mu,theta)

def sym_bias_over(c,mu,theta,thetatst,Ntst):
  return (thetatst**2)/(Ntst*sym_tau_over(c,mu,theta)**2)

def sym_bias_data_over(c,mu,M,Ntst):
  thetatst = sym.sqrt(Ntst)
  theta = sym.sqrt(M/c)
  return (thetatst**2)/(Ntst*sym_tau_over(c,mu,theta)**2)

def sym_wnorm_data_over(c,mu,M):
  theta = sym.sqrt(M/c)
  return sym_wnorm_over(c,mu,theta)/M

def sym_gen_error_opt_over(c,mu,Ntst,M):
  thetatst = sym.sqrt(Ntst)
  theta = optimal_theta_over(c,mu,thetatst,Ntst,M)
  return sym_bias_over(c,mu,theta,thetatst,Ntst) + sym_wnorm_over(c,mu,theta)/M

def sym_gen_error_over(c,mu,Ntst,M):
  thetatst = sym.sqrt(Ntst)
  theta = sym.sqrt(M/c)
  return sym_bias_over(c,mu,theta,thetatst,Ntst) + sym_wnorm_over(c,mu,theta)/M

def optimal_theta_over(c,mu,thetatst,Ntst,M):
  num = 2 * (thetatst**2 / Ntst) * sym_tnorm_over(c,mu) * sym_knorm_over(c,mu) * M - sym_hnorm_over(c,mu)
  denom = 2 * sym_tnorm_over(c,mu) ** 2 * sym_trace_term_over(c,mu) - sym_hnorm_over(c,mu) * sym_tnorm_over(c,mu) * sym_knorm_over(c,mu)
  return num / denom

In [ ]:
def sym_sim_lambda_inverse_M(c,mu):
  return (sym.sqrt((1+mu*c-c)**2 + 4*mu*c**2) -1 - mu*c + c)/(2*mu*c)

def sym_sim_lambda_inverse_N(c,mu):
  return (sym.sqrt(((c+mu*c-1)**2 + 4*mu*c)) + 1 - mu*c - c)/(2*mu)

def sym_sim_lambda_inverse_square_M(c,mu):
  term = sym.sqrt(4*mu*c**2 + (1-c+mu*c)**2)
  return (c**2 +mu*c**2 +c*mu - 2*c +1 )/(2*mu**2*c * term) + 1/(2*mu**2)*(1-1/c)

def sym_sim_lambda_inverse_square_N(c,mu):
  term = sym.sqrt(4*mu*c + (c-1+mu*c)**2)
  return (c**2 +mu*c**2 +c*mu - 2*c +1 )/(2*mu**2 * term) + 1/(2*mu**2)*(1-c)

def sym_sim_scale(c,mu):
  return 1/2 + (1+mu*c-sym.sqrt((1+mu*c-c)**2 + 4*mu*c**2))/(2*c)

def sym_sim_scale_squared(c,mu):
  return -1/2 + (1+c+mu*c)/(2*sym.sqrt((1+mu*c-c)**2 + 4*mu*c**2))

def sym_sim_scale_squared_over(c,mu):
  return c*(-1/2 + (1+c+mu*c)/(2*sym.sqrt((-1+mu*c-c)**2 + 4*mu*c)))

def sym_sim_scale_over(c,mu):
  return c/2 + (1+mu*c-sym.sqrt((-1+mu*c+c)**2 + 4*mu*c))/(2)

def sym_sim_hnorm(c,mu):
  return c * ((1+c+mu*c)/(2*sym.sqrt((1-c+c*mu)**2 + 4*c**2 * mu)) - 0.5)

def sym_sim_hnorm_over(c,mu):
  return c * ((1+c+mu*c)/(2*sym.sqrt((-1+c+c*mu)**2 + 4*c * mu)) - 0.5)

def sym_sim_knorm(c,mu):
  return (sym.sqrt((1+mu*c-c)**2 + 4*mu*c**2) -1 - mu*c + c)/(2*mu*c)

def sym_sim_knorm_over(c,mu):
  return (sym.sqrt((-1+mu*c+c)**2 + 4*mu*c) -1 - mu*c + c)/(2*mu*c)

def sym_sim_tnorm(c,mu):
  return (1-mu*c-c+sym.sqrt((1+mu*c-c)**2 + 4*mu*c**2))/2

def sym_sim_tnorm_over(c,mu):
  return (1-mu*c-c+sym.sqrt((-1+mu*c+c)**2 + 4*mu*c))/2

def sym_sim_tau(c,mu,theta):
  return 1 + theta**2 * (1+c+mu*c - sym.sqrt((1-c+mu*c)**2 + 4*mu*c**2))/2

def sym_sim_tau_over(c,mu,theta):
  return 1 + theta**2 * (1+c+mu*c - sym.sqrt((-1+c+mu*c)**2 + 4*mu*c))/2

def sym_sim_trace_term(c,mu):
  term = sym.sqrt(4*mu*c**2 + (1-c+mu*c)**2)
  return (c**2 +mu*c**2 +c*mu - 2*c +1 )/(2*mu**2*c * term) + 1/(2*mu**2)*(1-1/c)

def sym_sim_trace_term_over(c,mu):
  term = sym.sqrt(4*mu*c + (-1+c+mu*c)**2)
  return (c**2 +mu*c**2 +c*mu - 2*c +1 )/(2*mu**2*c * term) + (1-1/c)*(1/(2*mu**2))

def sym_sim_var(c,mu,theta):
  return (c*theta**2)*(theta**2+1)*((mu*c+c+1)/sym.sqrt((1-c+mu*c)**2 + 4*mu*c**2)-1)/2

def sym_sim_var_over(c,mu,theta):
  return (c*theta**2)*(theta**2+1)*((mu*c+c+1)/sym.sqrt((-1+c+mu*c)**2 + 4*mu*c)-1)/2

def sym_sim_gen_error(c,mu,theta,M):
  return (1 + sym_sim_var(c,mu,theta)/M)/(sym_sim_tau(c,mu,theta)**2)

def sym_sim_scale_both_squared(c,mu):
  return sym_sim_scale(c,mu) - (mu)*sym_sim_scale_squared(c,mu) 

def sym_sim_term1_training_error(c,mu,theta):
  return c * sym_sim_scale_both_squared(c,mu)*(theta**2)/(sym_sim_tau(c,mu,theta)**2)

def sym_sim_term2_training_error(c,mu,theta):
  return (sym_sim_tnorm(c,mu)**2) * sym_sim_scale_squared(c,mu)*(theta**4)/(sym_sim_tau(c,mu,theta)**2)
  
def sym_sim_wnorm_train(c,mu,theta):
  return sym_sim_term1_training_error(c,mu,theta) + sym_sim_term2_training_error(c,mu,theta)

def sym_sim_training_trace_term(c,mu,theta):
  return ((theta**2) * c * sym_sim_scale(c,mu) / sym_sim_tau(c,mu,theta)) - sym_sim_trace2(c,mu,theta)

def sym_sim_const_term(c,mu,theta):
  return theta ** 2 / (sym_sim_tau(c,mu,theta)**2)

def sym_sim_trace2(c,mu,theta):
  return c * (theta**4) * sym_sim_knorm(c,mu) * sym_sim_tnorm(c,mu) * sym_sim_scale(c,mu) / (sym_sim_tau(c,mu,theta)**2)

def sym_sim_training_error(c,mu,M):
  N = M/c
  theta = sym.sqrt(M/c)
  return (sym_sim_const_term(c,mu,theta) + sym_sim_wnorm_train(c,mu,theta) - 2*sym_sim_training_trace_term(c,mu,theta))/N

def sym_sim_training_error_param(c,mu,N):
  M = N*c
  theta = sym.sqrt(N)
  return (sym_sim_const_term(c,mu,theta) + sym_sim_wnorm_train(c,mu,theta) - 2*sym_sim_training_trace_term(c,mu,theta))/N  

def sym_sim_optimal_theta(c,mu,thetatst,Ntst,M):
  num = 2 * (thetatst**2 / Ntst) * sym_sim_tnorm(c,mu) * sym_sim_knorm(c,mu) * M - sym_sim_hnorm(c,mu)
  denom = 2 * sym_sim_tnorm(c,mu) ** 2 * sym_sim_trace_term(c,mu) - sym_sim_hnorm(c,mu) * sym_sim_tnorm(c,mu) * sym_sim_knorm(c,mu)
  return num / denom

In [ ]:
import sys
sys.setrecursionlimit(2000)
sys.getrecursionlimit()

2000

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Experiment for computing and plotting the derivative of the training error

In [ ]:
f = sym.lambdify([c,mu,M], sym_sim_training_error(c,mu,M), "numpy")
df = sym.lambdify([c,mu,M], sym_sim_training_error(c,mu,M).diff(c), "numpy")
ddf = sym.lambdify([c,mu,M], sym_sim_training_error(c,mu,M).diff(c).diff(c), "numpy")
dddf = sym.lambdify([c,mu,M], sym_sim_training_error(c,mu,M).diff(c).diff(c).diff(c), "numpy")
# ddddf = sym.lambdify([c,mu,M], sym_sim_training_error(c,mu,M).diff(c).diff(c).diff(c).diff(c), "numpy")
# dddddf = sym.lambdify([c,mu,M], sym_sim_training_error(c,mu,M).diff(c).diff(c).diff(c).diff(c).diff(c), "numpy")

In [ ]:
from tqdm import tqdm

# f = sym_sim_training_error(c,mu,N)
# df = sym_sim_training_error(c,mu,N).diff(c)
# ddf = sym_sim_training_error(c,mu,N).diff(c).diff(c)
# dddf = sym_sim_training_error(c,mu,N).diff(c).diff(c).diff(c)

mu_values = torch.linspace(0.04,10,100)
c_values = torch.linspace(0.005,1,500)

values = torch.zeros(4,100,500)
M_value = 1000

for i in tqdm(list(range(100))):
  mu_value = mu_values[i]
  for j,c_value in enumerate(c_values):
    values[0,i,j] = f(c_value,mu_value,M_value) #float(f.evalf(subs={c:c_value,mu:mu_value,N:M_value}))
    values[1,i,j] = df(c_value,mu_value,M_value) #float(df.evalf(subs={c:c_value,mu:mu_value,N:M_value}))
    values[2,i,j] = ddf(c_value,mu_value,M_value) #float(ddf.evalf(subs={c:c_value,mu:mu_value,N:M_value}))
    values[3,i,j] = dddf(c_value,mu_value,M_value) #float(dddf.evalf(subs={c:c_value,mu:mu_value,N:M_value}))

torch.save(values,"drive/MyDrive/derivative_data-more-mu.pt")

100%|██████████| 100/100 [1:57:55<00:00, 70.75s/it]


In [ ]:
f = sym.lambdify([c,mu,N], sym_sim_training_error_param(c,mu,N), "numpy")
df = sym.lambdify([c,mu,N], sym_sim_training_error_param(c,mu,N).diff(c), "numpy")
ddf = sym.lambdify([c,mu,N], sym_sim_training_error_param(c,mu,N).diff(c).diff(c), "numpy")
dddf = sym.lambdify([c,mu,N], sym_sim_training_error_param(c,mu,N).diff(c).diff(c).diff(c), "numpy")


mu_values = torch.linspace(0.04,10,100)
c_values = torch.linspace(0.005,1,500)

values = torch.zeros(4,100,500)
N_value = 1000

for i in tqdm(list(range(100))):
  mu_value = mu_values[i]
  for j,c_value in enumerate(c_values):
    values[0,i,j] = f(c_value,mu_value,N_value) #float(f.evalf(subs={c:c_value,mu:mu_value,N:M_value}))
    values[1,i,j] = df(c_value,mu_value,N_value) #float(df.evalf(subs={c:c_value,mu:mu_value,N:M_value}))
    values[2,i,j] = ddf(c_value,mu_value,N_value) #float(ddf.evalf(subs={c:c_value,mu:mu_value,N:M_value}))
    values[3,i,j] = dddf(c_value,mu_value,N_value) #float(dddf.evalf(subs={c:c_value,mu:mu_value,N:M_value}))

torch.save(values,"drive/MyDrive/derivative_parameter-more-mu.pt")

100%|██████████| 100/100 [59:22<00:00, 35.63s/it]


# Calculations for the Proof of Theorem 2

In [ ]:
theta = sym.sqrt(M/c)
# mu = mu**2

In [ ]:
expr = sym_sim_gen_error(c,mu,theta,M).diff(c).simplify()
expr

2*(2*d*(-2*c*sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2) + (c + d)*(-c*mu**2 - c + sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2) - 1))*(c*(-4*c*mu**2 - (mu**2 - 1)*(c*mu**2 - c + 1) + (mu**2 + 1)*sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2)) - sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2)*(c*mu**2 + c - sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2) + 1))*(4*c**2*mu**2 + (c*mu**2 - c + 1)**2)**(5/2) + (2*c + d*(c*mu**2 + c - sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2) + 1))*(4*c**2*mu**2 + (c*mu**2 - c + 1)**2)**2*(c*(c + d)*((mu**2 + 1)*(4*c**2*mu**2 + (c*mu**2 - c + 1)**2) - (4*c*mu**2 + (mu**2 - 1)*(c*mu**2 - c + 1))*(c*mu**2 + c + 1)) + d*(4*c**2*mu**2 + (c*mu**2 - c + 1)**2)*(-c*mu**2 - c + sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2) - 1)))/((2*c + d*(c*mu**2 + c - sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2) + 1))**3*(4*c**2*mu**2 + (c*mu**2 - c + 1)**2)**(7/2))

In [ ]:
print(sym.latex(sym_sim_gen_error(c,mu,theta,M).diff(c)))

\frac{\frac{\left(1 + \frac{d}{c}\right) \left(\frac{\mu^{2} + 1}{\sqrt{4 c^{2} \mu^{2} + \left(c \mu^{2} - c + 1\right)^{2}}} + \frac{\left(- 4 c \mu^{2} - \frac{\left(2 \mu^{2} - 2\right) \left(c \mu^{2} - c + 1\right)}{2}\right) \left(c \mu^{2} + c + 1\right)}{\left(4 c^{2} \mu^{2} + \left(c \mu^{2} - c + 1\right)^{2}\right)^{\frac{3}{2}}}\right)}{2} - \frac{d \left(-1 + \frac{c \mu^{2} + c + 1}{\sqrt{4 c^{2} \mu^{2} + \left(c \mu^{2} - c + 1\right)^{2}}}\right)}{2 c^{2}}}{\left(1 + \frac{d \left(c \mu^{2} + c - \sqrt{4 c^{2} \mu^{2} + \left(c \mu^{2} - c + 1\right)^{2}} + 1\right)}{2 c}\right)^{2}} + \frac{\left(\frac{\left(-1 + \frac{c \mu^{2} + c + 1}{\sqrt{4 c^{2} \mu^{2} + \left(c \mu^{2} - c + 1\right)^{2}}}\right) \left(1 + \frac{d}{c}\right)}{2} + 1\right) \left(- \frac{d \left(\mu^{2} - \frac{4 c \mu^{2} + \frac{\left(2 \mu^{2} - 2\right) \left(c \mu^{2} - c + 1\right)}{2}}{\sqrt{4 c^{2} \mu^{2} + \left(c \mu^{2} - c + 1\right)^{2}}} + 1\right)}{c} + \frac{d \left(c \mu^{2}

In [ ]:
sym.limit(expr,c,0).simplify()

1/(d + 1)

In [ ]:
expr2 = sym.limit(expr,c,1)
expr2

2*(-2*d**2*mu**16 + 2*d**2*mu**15*sqrt(mu**2 + 4) - 28*d**2*mu**14 + 24*d**2*mu**13*sqrt(mu**2 + 4) - 146*d**2*mu**12 + 102*d**2*mu**11*sqrt(mu**2 + 4) - 340*d**2*mu**10 + 176*d**2*mu**9*sqrt(mu**2 + 4) - 320*d**2*mu**8 + 96*d**2*mu**7*sqrt(mu**2 + 4) - 64*d**2*mu**6 - 2*d*mu**14 + 2*d*mu**13*sqrt(mu**2 + 4) - 26*d*mu**12 + 30*d*mu**11*sqrt(mu**2 + 4) - 120*d*mu**10 + 144*d*mu**9*sqrt(mu**2 + 4) - 224*d*mu**8 + 224*d*mu**7*sqrt(mu**2 + 4) - 128*d*mu**6 - 4*mu**10 - 32*mu**8 - 64*mu**6)/((mu**4 + 4*mu**2)**(7/2)*(d*mu**2 - d*mu*sqrt(mu**2 + 4) + 2*d + 2)**3)

In [ ]:
print(sym.latex(expr2))

\frac{2 \left(- 2 d^{2} \mu^{16} + 2 d^{2} \mu^{15} \sqrt{\mu^{2} + 4} - 28 d^{2} \mu^{14} + 24 d^{2} \mu^{13} \sqrt{\mu^{2} + 4} - 146 d^{2} \mu^{12} + 102 d^{2} \mu^{11} \sqrt{\mu^{2} + 4} - 340 d^{2} \mu^{10} + 176 d^{2} \mu^{9} \sqrt{\mu^{2} + 4} - 320 d^{2} \mu^{8} + 96 d^{2} \mu^{7} \sqrt{\mu^{2} + 4} - 64 d^{2} \mu^{6} - 2 d \mu^{14} + 2 d \mu^{13} \sqrt{\mu^{2} + 4} - 26 d \mu^{12} + 30 d \mu^{11} \sqrt{\mu^{2} + 4} - 120 d \mu^{10} + 144 d \mu^{9} \sqrt{\mu^{2} + 4} - 224 d \mu^{8} + 224 d \mu^{7} \sqrt{\mu^{2} + 4} - 128 d \mu^{6} - 4 \mu^{10} - 32 \mu^{8} - 64 \mu^{6}\right)}{\left(\mu^{4} + 4 \mu^{2}\right)^{\frac{7}{2}} \left(d \mu^{2} - d \mu \sqrt{\mu^{2} + 4} + 2 d + 2\right)^{3}}


In [ ]:
num,den = sym.fraction(expr2)
num.collect(M)

d**2*(-4*mu**16 + 4*mu**15*sqrt(mu**2 + 4) - 56*mu**14 + 48*mu**13*sqrt(mu**2 + 4) - 292*mu**12 + 204*mu**11*sqrt(mu**2 + 4) - 680*mu**10 + 352*mu**9*sqrt(mu**2 + 4) - 640*mu**8 + 192*mu**7*sqrt(mu**2 + 4) - 128*mu**6) + d*(-4*mu**14 + 4*mu**13*sqrt(mu**2 + 4) - 52*mu**12 + 60*mu**11*sqrt(mu**2 + 4) - 240*mu**10 + 288*mu**9*sqrt(mu**2 + 4) - 448*mu**8 + 448*mu**7*sqrt(mu**2 + 4) - 256*mu**6) - 8*mu**10 - 64*mu**8 - 128*mu**6

# Computation for Proof of Theorem 3


In [ ]:
T = sym.Symbol('T')

In [ ]:
num,den = sym.fraction(expr)
print(sym.latex(sym.factor(num).replace(sym.sqrt(c**2*mu**2+2*c**2*mu+c**2+2*c*mu-2*c+1),T)))

- 4 \left(c^{2} \mu^{4} + 2 c^{2} \mu^{2} + c^{2} + 2 c \mu^{2} - 2 c + 1\right)^{2} \left(- T c^{3} d^{2} \mu^{6} - 3 T c^{3} d^{2} \mu^{4} - 3 T c^{3} d^{2} \mu^{2} - T c^{3} d^{2} - T c^{3} d \mu^{4} - 5 T c^{3} d \mu^{2} - 4 T c^{3} d - T c^{2} d^{2} \mu^{4} - T c^{2} d^{2} \mu^{2} - 2 T c^{2} d \mu^{2} + 5 T c^{2} d + T c d^{2} \mu^{2} - T c d + T d^{2} + c^{4} d^{2} \mu^{8} + 4 c^{4} d^{2} \mu^{6} + 6 c^{4} d^{2} \mu^{4} + 4 c^{4} d^{2} \mu^{2} + c^{4} d^{2} + c^{4} d \mu^{6} + 2 c^{4} d \mu^{4} + c^{4} d \mu^{2} + 2 c^{4} \mu^{2} + 2 c^{4} + 2 c^{3} d^{2} \mu^{6} + 3 c^{3} d^{2} \mu^{4} - c^{3} d^{2} + 3 c^{3} d \mu^{4} + 5 c^{3} d - 2 c^{3} + 3 c^{2} d \mu^{2} - 6 c^{2} d - 2 c d^{2} \mu^{2} + c d^{2} + c d - d^{2}\right)


In [ ]:
sym.factor(num).replace(sym.sqrt(c**2*mu**2+2*c**2*mu+c**2+2*c*mu-2*c+1),T)

-4*(c**2*mu**4 + 2*c**2*mu**2 + c**2 + 2*c*mu**2 - 2*c + 1)**2*(-T*c**3*d**2*mu**6 - 3*T*c**3*d**2*mu**4 - 3*T*c**3*d**2*mu**2 - T*c**3*d**2 - T*c**3*d*mu**4 - 5*T*c**3*d*mu**2 - 4*T*c**3*d - T*c**2*d**2*mu**4 - T*c**2*d**2*mu**2 - 2*T*c**2*d*mu**2 + 5*T*c**2*d + T*c*d**2*mu**2 - T*c*d + T*d**2 + c**4*d**2*mu**8 + 4*c**4*d**2*mu**6 + 6*c**4*d**2*mu**4 + 4*c**4*d**2*mu**2 + c**4*d**2 + c**4*d*mu**6 + 2*c**4*d*mu**4 + c**4*d*mu**2 + 2*c**4*mu**2 + 2*c**4 + 2*c**3*d**2*mu**6 + 3*c**3*d**2*mu**4 - c**3*d**2 + 3*c**3*d*mu**4 + 5*c**3*d - 2*c**3 + 3*c**2*d*mu**2 - 6*c**2*d - 2*c*d**2*mu**2 + c*d**2 + c*d - d**2)

In [ ]:
print(sym.latex(sym.factor(den).replace(sym.sqrt(c**2*mu**2+2*c**2*mu+c**2+2*c*mu-2*c+1),T)))

\left(- T d + c d \mu^{2} + c d + 2 c + d\right)^{3} \left(c^{2} \mu^{4} + 2 c^{2} \mu^{2} + c^{2} + 2 c \mu^{2} - 2 c + 1\right)^{\frac{7}{2}}


In [ ]:
sym.factor(den).replace(sym.sqrt(c**2*mu**2+2*c**2*mu+c**2+2*c*mu-2*c+1),T)

(-T*d + c*d*mu**2 + c*d + 2*c + d)**3*(c**2*mu**4 + 2*c**2*mu**2 + c**2 + 2*c*mu**2 - 2*c + 1)**(7/2)

In [ ]:
print(sym.latex(sym.sqrt(c**2*mu**2+2*c**2*mu+c**2+2*c*mu-2*c+1)))

\sqrt{c^{2} \mu^{4} + 2 c^{2} \mu^{2} + c^{2} + 2 c \mu^{2} - 2 c + 1}


In [ ]:
expr3 = sym.factor(num).replace(sym.sqrt(c**2*mu**2+2*c**2*mu+c**2+2*c*mu-2*c+1),T)
expr4 = sym.factor_list(expr3)[1][1][0]
expr4

-T*c**3*d**2*mu**6 - 3*T*c**3*d**2*mu**4 - 3*T*c**3*d**2*mu**2 - T*c**3*d**2 - T*c**3*d*mu**4 - 5*T*c**3*d*mu**2 - 4*T*c**3*d - T*c**2*d**2*mu**4 - T*c**2*d**2*mu**2 - 2*T*c**2*d*mu**2 + 5*T*c**2*d + T*c*d**2*mu**2 - T*c*d + T*d**2 + c**4*d**2*mu**8 + 4*c**4*d**2*mu**6 + 6*c**4*d**2*mu**4 + 4*c**4*d**2*mu**2 + c**4*d**2 + c**4*d*mu**6 + 2*c**4*d*mu**4 + c**4*d*mu**2 + 2*c**4*mu**2 + 2*c**4 + 2*c**3*d**2*mu**6 + 3*c**3*d**2*mu**4 - c**3*d**2 + 3*c**3*d*mu**4 + 5*c**3*d - 2*c**3 + 3*c**2*d*mu**2 - 6*c**2*d - 2*c*d**2*mu**2 + c*d**2 + c*d - d**2

In [ ]:
q,r = sym.div(expr4 - 4*M*mu*c**2*(2*mu*c-T), mu*c+c-1)

In [ ]:
q

-T*c**2*d**2*mu**4 - 2*T*c**2*d**2*mu**2 - T*c**2*d**2 - T*c**2*d*mu**2 - 4*T*c**2*d - 2*T*c*d**2*mu**2 - T*c*d**2 + T*c*d - T*d**2 + c**3*d**2*mu**6 + 3*c**3*d**2*mu**4 + 3*c**3*d**2*mu**2 + c**3*d**2 + c**3*d*mu**4 + c**3*d*mu**2 + 2*c**3 + 3*c**2*d**2*mu**4 + 3*c**2*d**2*mu**2 - 4*c**2*d*mu**2 + 5*c**2*d + 3*c*d**2*mu**2 - c*d + d**2

In [ ]:
print(sym.latex(q))

- T c^{2} d^{2} \mu^{4} - 2 T c^{2} d^{2} \mu^{2} - T c^{2} d^{2} - T c^{2} d \mu^{2} - 4 T c^{2} d - 2 T c d^{2} \mu^{2} - T c d^{2} + T c d - T d^{2} + c^{3} d^{2} \mu^{6} + 3 c^{3} d^{2} \mu^{4} + 3 c^{3} d^{2} \mu^{2} + c^{3} d^{2} + c^{3} d \mu^{4} + c^{3} d \mu^{2} + 2 c^{3} + 3 c^{2} d^{2} \mu^{4} + 3 c^{2} d^{2} \mu^{2} - 4 c^{2} d \mu^{2} + 5 c^{2} d + 3 c d^{2} \mu^{2} - c d + d^{2}


In [ ]:
r

0

# Computation for Proof of Theorem 4

In [87]:
theta = sym.Symbol('sigma')
expr = (sym_sim_var(c,mu,theta)/(sym_sim_tau(c,mu,theta)**2)).diff(c)
expr

-c*sigma**4*(-1 + (c*mu**2 + c + 1)/sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2))*(sigma**2 + 1)*(mu**2 - (4*c*mu**2 + (2*mu**2 - 2)*(c*mu**2 - c + 1)/2)/sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2) + 1)/(2*(sigma**2*(c*mu**2 + c - sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2) + 1)/2 + 1)**3) + c*sigma**2*(sigma**2 + 1)*((mu**2 + 1)/sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2) + (-4*c*mu**2 - (2*mu**2 - 2)*(c*mu**2 - c + 1)/2)*(c*mu**2 + c + 1)/(4*c**2*mu**2 + (c*mu**2 - c + 1)**2)**(3/2))/(2*(sigma**2*(c*mu**2 + c - sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2) + 1)/2 + 1)**2) + sigma**2*(-1 + (c*mu**2 + c + 1)/sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2))*(sigma**2 + 1)/(2*(sigma**2*(c*mu**2 + c - sqrt(4*c**2*mu**2 + (c*mu**2 - c + 1)**2) + 1)/2 + 1)**2)

In [90]:
expr = expr.simplify()

In [88]:
print(sym.latex(expr))

- \frac{c \sigma^{4} \left(-1 + \frac{c \mu^{2} + c + 1}{\sqrt{4 c^{2} \mu^{2} + \left(c \mu^{2} - c + 1\right)^{2}}}\right) \left(\sigma^{2} + 1\right) \left(\mu^{2} - \frac{4 c \mu^{2} + \frac{\left(2 \mu^{2} - 2\right) \left(c \mu^{2} - c + 1\right)}{2}}{\sqrt{4 c^{2} \mu^{2} + \left(c \mu^{2} - c + 1\right)^{2}}} + 1\right)}{2 \left(\frac{\sigma^{2} \left(c \mu^{2} + c - \sqrt{4 c^{2} \mu^{2} + \left(c \mu^{2} - c + 1\right)^{2}} + 1\right)}{2} + 1\right)^{3}} + \frac{c \sigma^{2} \left(\sigma^{2} + 1\right) \left(\frac{\mu^{2} + 1}{\sqrt{4 c^{2} \mu^{2} + \left(c \mu^{2} - c + 1\right)^{2}}} + \frac{\left(- 4 c \mu^{2} - \frac{\left(2 \mu^{2} - 2\right) \left(c \mu^{2} - c + 1\right)}{2}\right) \left(c \mu^{2} + c + 1\right)}{\left(4 c^{2} \mu^{2} + \left(c \mu^{2} - c + 1\right)^{2}\right)^{\frac{3}{2}}}\right)}{2 \left(\frac{\sigma^{2} \left(c \mu^{2} + c - \sqrt{4 c^{2} \mu^{2} + \left(c \mu^{2} - c + 1\right)^{2}} + 1\right)}{2} + 1\right)^{2}} + \frac{\sigma^{2} \left(-1 + \f

In [91]:
sym.limit(expr,c,1/(mu+1))

2*sigma**2*(mu**2 + 1)**(3/2)*(-256*mu**7 + 256*mu**6*sqrt(mu**2 + 1))*(sigma**2 + 1)/((4*mu**4/(mu**2 + 1)**2 + 4*mu**2/(mu**2 + 1)**2)**(7/2)*(-2*mu*sigma**2 + 2*sigma**2*sqrt(mu**2 + 1) + 2*sqrt(mu**2 + 1))**3*(mu**6*sqrt(mu**2 + 1) + 3*mu**4*sqrt(mu**2 + 1) + 3*mu**2*sqrt(mu**2 + 1) + sqrt(mu**2 + 1)))

In [92]:
print(sym.latex(sym.limit(expr,c,1/(mu+1))))

\frac{2 \sigma^{2} \left(\mu^{2} + 1\right)^{\frac{3}{2}} \left(- 256 \mu^{7} + 256 \mu^{6} \sqrt{\mu^{2} + 1}\right) \left(\sigma^{2} + 1\right)}{\left(\frac{4 \mu^{4}}{\left(\mu^{2} + 1\right)^{2}} + \frac{4 \mu^{2}}{\left(\mu^{2} + 1\right)^{2}}\right)^{\frac{7}{2}} \left(- 2 \mu \sigma^{2} + 2 \sigma^{2} \sqrt{\mu^{2} + 1} + 2 \sqrt{\mu^{2} + 1}\right)^{3} \left(\mu^{6} \sqrt{\mu^{2} + 1} + 3 \mu^{4} \sqrt{\mu^{2} + 1} + 3 \mu^{2} \sqrt{\mu^{2} + 1} + \sqrt{\mu^{2} + 1}\right)}


In [93]:
sym.limit(expr,c,1).collect(theta)

2*sigma**2*(sigma**2 + 1)*(2*mu**14 - 2*mu**13*sqrt(mu**2 + 4) + 28*mu**12 - 24*mu**11*sqrt(mu**2 + 4) + 140*mu**10 - 96*mu**9*sqrt(mu**2 + 4) + 288*mu**8 - 128*mu**7*sqrt(mu**2 + 4) + 192*mu**6 + sigma**2*(-2*mu**16 + 2*mu**15*sqrt(mu**2 + 4) - 28*mu**14 + 24*mu**13*sqrt(mu**2 + 4) - 146*mu**12 + 102*mu**11*sqrt(mu**2 + 4) - 340*mu**10 + 176*mu**9*sqrt(mu**2 + 4) - 320*mu**8 + 96*mu**7*sqrt(mu**2 + 4) - 64*mu**6))/((mu**4 + 4*mu**2)**(7/2)*(sigma**2*(mu**2 - mu*sqrt(mu**2 + 4) + 2) + 2)**3)

In [96]:
print(sym.latex(sym.limit(expr,c,1).collect(theta)))

\frac{2 \sigma^{2} \left(\sigma^{2} + 1\right) \left(2 \mu^{14} - 2 \mu^{13} \sqrt{\mu^{2} + 4} + 28 \mu^{12} - 24 \mu^{11} \sqrt{\mu^{2} + 4} + 140 \mu^{10} - 96 \mu^{9} \sqrt{\mu^{2} + 4} + 288 \mu^{8} - 128 \mu^{7} \sqrt{\mu^{2} + 4} + 192 \mu^{6} + \sigma^{2} \left(- 2 \mu^{16} + 2 \mu^{15} \sqrt{\mu^{2} + 4} - 28 \mu^{14} + 24 \mu^{13} \sqrt{\mu^{2} + 4} - 146 \mu^{12} + 102 \mu^{11} \sqrt{\mu^{2} + 4} - 340 \mu^{10} + 176 \mu^{9} \sqrt{\mu^{2} + 4} - 320 \mu^{8} + 96 \mu^{7} \sqrt{\mu^{2} + 4} - 64 \mu^{6}\right)\right)}{\left(\mu^{4} + 4 \mu^{2}\right)^{\frac{7}{2}} \left(\sigma^{2} \left(\mu^{2} - \mu \sqrt{\mu^{2} + 4} + 2\right) + 2\right)^{3}}
